In [91]:
import pandas as pd
import datetime
from tqdm.auto import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


source: https://www.kaggle.com/oalvay/enron-emails-complete-preprocessing?select=emails_cleaned.csv 

In [92]:
emails_cleaned = pd.read_csv('emails_cleaned.csv')
emails_cleaned = emails_cleaned[["Date", "From", "To"]]

def str2datetime(Date):
    Date = Date.split("(")[0]
    date_time_obj = datetime.datetime.strptime(Date, '%a, %d %b %Y %H:%M:%S %z ')
    return date_time_obj.date()

dates = emails_cleaned['Date'].apply(str2datetime)
# dates = pd.to_datetime(dates, infer_datetime_format=True)


In [93]:
emails_cleaned.insert(0,'Dates',dates)
emails_cleaned = emails_cleaned.drop(columns = ['Date'])
emails_cleaned = emails_cleaned.dropna()
emails_cleaned = emails_cleaned.drop_duplicates()
emails_cleaned = emails_cleaned.sort_values(by=['Dates'])

In [94]:
df_checkaddrnum = pd.DataFrame(columns=['Dates', 'From', 'To'])
def checkaddrnum(row):
    global df_checkaddrnum
    # df_checkaddrnum = pd.DataFrame(columns=['Dates', 'From', 'To'])
    if ' ' in row['To']:
        temp = row['To'].split(',')

        for item in temp:
            df_checkaddrnum = df_checkaddrnum.append({'Dates': row['Dates'], 'From': row['From'], 'To': item.strip()}, ignore_index=True)
        # return df_checkaddrnum = pd.DataFrame(columns=['Dates', 'From', 'To'])
def checkcomma(row):
    if ',' in row: return True
    else: return False

In [95]:
emails_cleaned.parallel_apply(checkaddrnum, axis=1)
emails_cleaned.insert(3, 'Checkcomma', emails_cleaned['To'].parallel_apply(checkcomma))
emails_cleaned.drop(emails_cleaned[emails_cleaned['Checkcomma'] == True].index, inplace = True)
emails_cleaned = emails_cleaned.append(df_checkaddrnum)
emails_cleaned = emails_cleaned.drop(columns = 'Checkcomma')
emails_cleaned = emails_cleaned.sort_values(by=['Dates'])


In [101]:
emails_cleaned

,Dates,From,To
443793,0001-05-30,payables.ibuyit@enron.co,jeff.skilling@enron.co
342330,0001-06-07,payables.ibuyit@enron.co,scott.neal@enron.co
390578,0001-06-18,payables.ibuyit@enron.co,holden.salisbury@enron.co
391543,0001-07-24,payables.ibuyit@enron.co,holden.salisbury@enron.co
288078,0001-08-01,payables.ibuyit@enron.co,michelle.lokay@enron.co
...,...,...,...
174655,2007-02-11,rwalker802@earthlink.ne,kevin.hyatt@enron.co
43809,2020-12-24,jody421@hotmail.co,undisclosed.recipients@mailman.enron.co
43852,2020-12-29,jody421@hotmail.co,undisclosed.recipients@mailman.enron.co
517038,2044-01-02,cramer@cadvision.co,john.zufferli@enron.co


In [104]:
emails_cleaned.to_csv(r'./emails_cleaned_Eric.csv', index = False)